Put csv file name and airline id str as variables in the function below.\
Then you can get reply time results.

In [3]:
import pandas as pd
from ast import literal_eval
import datetime
def time_calculate(csv_name, user_id):
    result = pd.read_csv(csv_name)
    result[['tweet_created_at', 'id_str', 'entity_user_mentions','full_conversation_ids']] = result[['tweet_created_at', 'id_str', 'entity_user_mentions','full_conversation_ids']].astype(str)
    all_result = pd.read_csv('result.csv', index_col=0)#Read all Tweet data.
    all_result[['in_reply_to_user_id_str','in_reply_to_status_id_str', 'entity_user_mentions', 'id_str', 'user_id_str']] = all_result[['in_reply_to_user_id_str','in_reply_to_status_id_str', 'entity_user_mentions', 'id_str', 'user_id_str']].astype(str)
    all_result = all_result[['tweet_created_at', 'id_str','user_id_str']]
    result['full_conversation_ids'] = result['full_conversation_ids'].apply(lambda x:literal_eval(str(x)))
    result['con_id_str_list1'] = result['con_id_str_list1'].apply(lambda x:literal_eval(str(x)))
    result = result[result['full_conversation_ids'].str.len() >= 2]
    #Create dict for create time and user id.Select conversations contain airline reply. 
    time_dict = dict(zip(all_result['id_str'], all_result['tweet_created_at']))
    user_dict = dict(zip(all_result['id_str'], all_result['user_id_str']))
    list_56377143 = all_result[all_result['user_id_str']==user_id]['id_str'].tolist()
    result['airline_contain_con'] = result['full_conversation_ids'].apply(lambda x:list(set(x).intersection(set(list_56377143))))
    result = result[result['airline_contain_con'].str.len() >= 1]#found all conversations have airline reply.
    result['airline_contain_con_time'] = result['airline_contain_con'].apply(lambda x:[time_dict[i] for i in x])
    result['airline_reply_time'] = result['airline_contain_con_time'].apply(lambda x:x[0])#first reply time
    result['airline_reply_time']=pd.to_datetime(result['airline_reply_time'],format='%Y-%m-%d %H:%M:%S.%f')
    result['tweet_created_at']=pd.to_datetime(result['tweet_created_at'],format='%Y-%m-%d %H:%M:%S.%f')
    result[user_id] = result['airline_reply_time']-result['tweet_created_at']
    result[user_id] = result[user_id].dt.total_seconds()/60
    KLM_time = result[[user_id]].describe().astype(str)
    KLM_time.to_excel(f'{user_id}_time_describe.xls')
    result.to_csv(f'{user_id}_time.csv')
    print(KLM_time)
    return None
time_calculate('conversations_124476322', '124476322')

/Users/shishaoxuan/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
<ipython-input-3-b7097ef1fc1b>:26: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  KLM_time.to_excel(f'{user_id}_time_describe.xls')


                 124476322
count               1594.0
mean    23.814230447511534
std     108.02166339681845
min    0.38333333333333336
25%     3.3333333333333335
50%                    6.4
75%     12.879166666666666
max                 2583.0
